In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import userdata
wandb_token = userdata.get("WANDB_TOKEN")

!wandb login {wandb_token}

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

In [ ]:
# ⬇️  Run this cell by itself in Colab
from huggingface_hub import login
hf_token = userdata.get("HF_TOKEN")

# 1️⃣  Authenticate (generate a write-token at https://huggingface.co/settings/tokens)
login(hf_token)

In [ ]:
VERSION=14
SIZE="1.7B"
MODEL_NAME=f"unsloth/qwen3-{SIZE}"
COMMIT_MESSAGE="first commit"

In [ ]:
from unsloth import FastLanguageModel
import torch

fourbit_models = [
    "unsloth/Qwen3-1.7B-unsloth-bnb-4bit", # Qwen 14B 2x faster
    "unsloth/Qwen3-4B-unsloth-bnb-4bit",
    "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    "unsloth/Qwen3-14B-unsloth-bnb-4bit",
    "unsloth/Qwen3-32B-unsloth-bnb-4bit",

    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/Phi-4",
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/orpheus-3b-0.1-ft-unsloth-bnb-4bit" # [NEW] We support TTS models!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_NAME,
    max_seq_length = 5500,   # Context length - can be longer, but uses more memory
    load_in_4bit = True,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = False, # We have full finetuning now!
    # token = "hf_...",      # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.1: Fast Qwen3 patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.41G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [ ]:
lora_r = 16

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_r,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = lora_r * 2,  # Best to choose alpha = rank or rank*2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,   # We support rank stabilized LoRA
    loftq_config = None,  # And LoftQ
)

Unsloth 2025.12.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
tokenizer.eos_token

'<|im_end|>'

In [ ]:
from datasets import load_dataset, concatenate_datasets

In [ ]:
from jinja2 import Template

SYSTEM_PROMPT = Template("""\
{{ role }}

# Metadata

{{ metadata }}

# Tools

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{% for item in functions %}
{{- item }}
{% endfor %}</tools>

# Important

- Reply in 1-4 sentences
- Sound natural
- Respect your persona
- Show genuine curiosity
- Ask follow-up questions""")

In [ ]:
DATASET = "nuriyev/cpdc-agent"

In [ ]:
from huggingface_hub import HfApi

api = HfApi()
dataset_info = api.dataset_info(DATASET)
dataset_commit_hash = dataset_info.sha

print(f"Dataset commit hash: {dataset_commit_hash}")

Dataset commit hash: 50bb3fff24351af95aaab78bf04378d852fc3ef8


In [ ]:
import json
from datasets import Dataset

train_dataset = load_dataset(DATASET, split="train", token=True)
eval_dataset  = load_dataset(DATASET, split="test", token=True)

def preprocess(example):
    worldview = example["worldview"]
    persona = json.loads(example["persona"])
    knowledge = json.loads(example["knowledge"])
    state = json.loads(example["state"])
    functions = json.loads(example["functions"])
    messages = [
      {"role": "system", "content": SYSTEM_PROMPT.render(
          role=example["role"],
          metadata=json.dumps({
              "worldview": worldview,
              "persona": persona,
              "knowledge": knowledge,
              "state": state,
          }, ensure_ascii=False),
          functions=[json.dumps({"name": f["name"], "description": f["description"]}, ensure_ascii=False) for f in functions]
      )},
      *example["messages"]
    ]
    return {"messages": messages}

train_dataset = train_dataset.map(preprocess)
train_dataset = train_dataset.select_columns(["messages"])
eval_dataset  = eval_dataset.map(preprocess)
eval_dataset  = eval_dataset.select_columns(["messages"])

def convert_set_to_input_labels(dataset):
  new_dataset = []
  for conv in dataset:
      messages = [conv["messages"][0]]
      try:
        for i in range(1, len(conv["messages"]), 2):
            user_message = conv["messages"][i]
            assistant_message = conv["messages"][i + 1]
            messages.append(user_message)
            new_dataset.append(
                {
                    "prompt": messages.copy(),
                    "completion": [assistant_message],
                }
            )
            messages.append(assistant_message)
      except IndexError as e:
        print(e)
        print(conv)

  dataset = new_dataset
  new_dataset = []
  for conv in dataset:
    if len(conv["prompt"]) <= 2:
      new_dataset.append(conv)
      continue

    prompt = [conv["prompt"][0]]
    for p in conv["prompt"][1:-2]:
      if "<tool_call>" not in p['content'] and "<tool_response>" not in p['content']:
        prompt.append(p)
    prompt.append(conv["prompt"][-2])
    prompt.append(conv["prompt"][-1])
    new_dataset.append(
        {
            "prompt": prompt,
            "completion": conv["completion"],
        }
    )

  dataset = new_dataset
  new_dataset = []

  for conv in dataset:
    if "<tool_call>" in conv["completion"][-1]['content']:
      continue
    new_dataset.append(conv)


  return Dataset.from_list(new_dataset)

train_dataset = convert_set_to_input_labels(train_dataset)
eval_dataset  = convert_set_to_input_labels(eval_dataset)

In [ ]:
len(train_dataset)

1704

In [ ]:
print(train_dataset[401]['prompt'][0]['content'])

Play the role of a merchant selling weapons at a weapon shop. After selling a weapon, always confirm whether the customer wants to equip it.

# Metadata

{"worldview": "In a world overrun by monsters, humans protect themselves not only with weapons and armor but also by developing magical technologies to combat the monster invasions. They have formed guilds, uniting adventurers and bounty hunters to create organizations capable of fighting back against the monsters. Guild members work daily to maintain peace in the towns and to destroy the monsters' dens. Although the world is filled with tension, people have not lost their smiles. Towns are lively, filled with markets selling fresh food and the sounds of weapons, armor, and magical tools being crafted. At night, parties who have defeated the monsters gather to celebrate. The currency unit is gold. The currency symbol is written as Gold, gold, G, g. There are four difficulty level for quests: A, B, C, S. The level goes from easy to hig

In [ ]:
index = 102
{"prompt":train_dataset[index]['prompt'][1:], "completion": train_dataset[index]['completion']}

{'prompt': [{'content': "I'm looking for a sword. Something practical but within a budget of 1500 gold. What do you have?",
   'role': 'user'},
  {'content': "Ah, swords! I've got a few that fit your needs. Let's see... Would you prefer a single-handed or a double-handed sword?",
   'role': 'assistant'},
  {'content': "I don't have anything particular in mind. Show me everything you have. Let's start with the double-handed sword.",
   'role': 'user'},
  {'content': "The Two-Handed Sword is a fine piece. It's priced at 1200 gold, and it's attack level is 60. It is long, thick, and ideal for cutting down multiple foes.",
   'role': 'assistant'},
  {'content': 'That sounds promising. What about your single-handed options?',
   'role': 'user'},
  {'content': "The Short Sword is 300 gold. It's light and easy to handle with an attack level of 15.",
   'role': 'assistant'},
  {'content': 'Is that all the options you have? What other single-handed swords do you have to offer?',
   'role': 'use

In [ ]:
index = 103
{"prompt":train_dataset[index]['prompt'][1:], "completion": train_dataset[index]['completion']}

{'prompt': [{'content': "I'm looking for a sword. Something practical but within a budget of 1500 gold. What do you have?",
   'role': 'user'},
  {'content': "Ah, swords! I've got a few that fit your needs. Let's see... Would you prefer a single-handed or a double-handed sword?",
   'role': 'assistant'},
  {'content': "I don't have anything particular in mind. Show me everything you have. Let's start with the double-handed sword.",
   'role': 'user'},
  {'content': "The Two-Handed Sword is a fine piece. It's priced at 1200 gold, and it's attack level is 60. It is long, thick, and ideal for cutting down multiple foes.",
   'role': 'assistant'},
  {'content': 'That sounds promising. What about your single-handed options?',
   'role': 'user'},
  {'content': "The Short Sword is 300 gold. It's light and easy to handle with an attack level of 15.",
   'role': 'assistant'},
  {'content': 'Is that all the options you have? What other single-handed swords do you have to offer?',
   'role': 'use

In [ ]:
index = 104
{"prompt":train_dataset[index]['prompt'][1:], "completion": train_dataset[index]['completion']}

{'prompt': [{'content': "I'm looking for a sword. Something practical but within a budget of 1500 gold. What do you have?",
   'role': 'user'},
  {'content': "Ah, swords! I've got a few that fit your needs. Let's see... Would you prefer a single-handed or a double-handed sword?",
   'role': 'assistant'},
  {'content': "I don't have anything particular in mind. Show me everything you have. Let's start with the double-handed sword.",
   'role': 'user'},
  {'content': "The Two-Handed Sword is a fine piece. It's priced at 1200 gold, and it's attack level is 60. It is long, thick, and ideal for cutting down multiple foes.",
   'role': 'assistant'},
  {'content': 'That sounds promising. What about your single-handed options?',
   'role': 'user'},
  {'content': "The Short Sword is 300 gold. It's light and easy to handle with an attack level of 15.",
   'role': 'assistant'},
  {'content': 'Is that all the options you have? What other single-handed swords do you have to offer?',
   'role': 'use

In [ ]:
# assert that all completions are of "role":"assistant"
for conv in train_dataset:
    assert all(msg["role"] == "assistant" for msg in conv["completion"])

for conv in eval_dataset:
    assert all(msg["role"] == "assistant" for msg in conv["completion"])

In [ ]:
def preprocess(example):
    prompt_str = tokenizer.apply_chat_template(
        example['prompt'],
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    if not example["completion"]:
      print(example)

    full       = prompt_str + example["completion"][-1]["content"] + tokenizer.eos_token
    tokenized  = tokenizer(full, return_attention_mask=False)

    prompt_len = len(tokenizer(prompt_str, return_attention_mask=False)["input_ids"])
    labels     = [-100] * prompt_len + tokenized["input_ids"][prompt_len:]

    return {"input_ids": tokenized["input_ids"], "labels": labels}

train_dataset = train_dataset.map(preprocess)
eval_dataset  = eval_dataset.map(preprocess)

Map:   0%|          | 0/1704 [00:00<?, ? examples/s]

Map:   0%|          | 0/48 [00:00<?, ? examples/s]

In [ ]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(features):
    ids   = [torch.tensor(f["input_ids"]) for f in features]
    lbls  = [torch.tensor(f["labels"])    for f in features]

    ids_p   = pad_sequence(ids,  batch_first=True, padding_value=tokenizer.pad_token_id)
    lbls_p  = pad_sequence(lbls, batch_first=True, padding_value=-100)
    attn_ms = ids_p.ne(tokenizer.pad_token_id).long()

    return {"input_ids": ids_p,
            "labels": lbls_p,
            "attention_mask": attn_ms}


In [ ]:
OUTPUT_DIR = f"(...)/qwen-{SIZE}-finetuned_persona_v{VERSION}"

In [ ]:
from transformers import EarlyStoppingCallback
from trl import SFTTrainer, SFTConfig

callbacks = [EarlyStoppingCallback(
    early_stopping_patience = 5,   # 3 eval‑checks with no improvement
    early_stopping_threshold = 0.0 # need strictly better eval_loss
)]

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    data_collator=collate_fn,
    callbacks=callbacks,
    args = SFTConfig(
        num_train_epochs   = 2,   # set high, ES will stop before
        output_dir=OUTPUT_DIR,
        load_best_model_at_end = True,
        metric_for_best_model = "eval_loss",
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_ratio=0.15,
        warmup_steps=10,
        learning_rate = 1e-4,
        eval_strategy="steps",
        save_strategy="steps",
        logging_steps=30,
        save_steps=20,
        eval_steps=20,
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "wandb", # Use this for WandB etc
    ),
)

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA L4. Max memory = 22.161 GB.
1.697 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()
# trainer_stats = trainer.train(resume_from_checkpoint = True)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,704 | Num Epochs = 2 | Total steps = 214
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 17,432,576 of 1,738,007,552 (1.00% trained)


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
20,No log,2.003867
40,2.801000,1.883197
60,1.819700,1.839580
80,1.819700,1.813874
100,1.702500,1.797564
120,1.629100,1.785869
140,1.629100,1.754870
160,1.514800,1.762275
180,1.534800,1.764398
200,1.534800,1.756559


Unsloth: Not an error, but Qwen3ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


eval/loss,█▅▃▃▂▂▁▁▁▁
eval/runtime,█▁▁▁▁▁▁▁▁▁
eval/samples_per_second,▁█████████
eval/steps_per_second,▁█████████
train/epoch,▁▁▂▂▂▃▄▄▅▅▅▆▆▇▇███
train/global_step,▁▁▂▂▂▃▄▄▅▅▅▆▆▇▇▇██
train/grad_norm,▂▂▁▂▅▆█
train/learning_rate,█▇▆▄▃▂▁
train/loss,█▃▂▂▁▁▁
eval/loss,1.75656
eval/runtime,16.779


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

3227.4136 seconds used for training.
53.79 minutes used for training.
Peak reserved memory = 17.221 GB.
Peak reserved memory for training = 15.524 GB.
Peak reserved memory % of max memory = 77.709 %.
Peak reserved memory for training % of max memory = 70.051 %.


In [ ]:
del train_dataset, eval_dataset

In [ ]:
REPO_ID = f"nuriyev/qwen3-{SIZE}-cpdc-persona-lora"
model.push_to_hub(REPO_ID, tokenizer, save_method = "lora", token = hf_token, commit_message=COMMIT_MESSAGE)

README.md:   0%|          | 0.00/590 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   0%|          | 45.8kB / 69.8MB            

Saved model to https://huggingface.co/nuriyev/qwen3-1.7B-cpdc-persona-lora


In [ ]:
# REPO_ID = f"nuriyev/qwen3-{SIZE}-cpdc-persona"
# model.push_to_hub_merged(REPO_ID, tokenizer, save_method = "merged_16bit", token = hf_token, commit_message=COMMIT_MESSAGE)

config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...dc-persona/tokenizer.json:   0%|          | 28.3kB / 11.4MB            

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


...

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...persona/model.safetensors:   1%|1         | 41.8MB / 3.44GB            

Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [01:09<00:00, 69.00s/it]


Unsloth: Merge process complete. Saved to `/content/nuriyev/qwen3-1.7B-cpdc-persona`


In [ ]:
from huggingface_hub import HfApi

api = HfApi()

# Get the latest commit hash on the 'main' branch
commit_info = api.list_repo_commits(repo_id=REPO_ID, revision="main")
commit_hash = commit_info[0].commit_id

print(f"✅  Uploaded to https://huggingface.co/{REPO_ID}")
print(f"📌  Commit hash: {commit_hash}")
print(f"🔗  Full commit URL: {commit_info}")

✅  Uploaded to https://huggingface.co/nuriyev/qwen3-1.7B-cpdc-persona-lora
📌  Commit hash: 5817c8459bdf84352989ca7232ebbeb59a86e6d2
🔗  Full commit URL: [GitCommitInfo(commit_id='5817c8459bdf84352989ca7232ebbeb59a86e6d2', authors=['nuriyev'], created_at=datetime.datetime(2025, 12, 9, 22, 49, 9, tzinfo=datetime.timezone.utc), title='first commit (Trained with Unsloth)', message='\n\nUpload model trained with Unsloth 2x faster', formatted_title=None, formatted_message=None), GitCommitInfo(commit_id='104786dfa5cd001102e7e1433d1e6bb84bc258e4', authors=['nuriyev'], created_at=datetime.datetime(2025, 12, 9, 22, 48, 55, tzinfo=datetime.timezone.utc), title='Upload README.md with huggingface_hub', message='', formatted_title=None, formatted_message=None), GitCommitInfo(commit_id='7476eb20ae17db4b37061c62e5da05a6524e9362', authors=['nuriyev'], created_at=datetime.datetime(2025, 12, 9, 22, 48, 53, tzinfo=datetime.timezone.utc), title='initial commit', message='', formatted_title=None, formatted_m

In [ ]:
from huggingface_hub import HfApi

api = HfApi()

# Get the base model name from previous cells
base_model_name = MODEL_NAME

# Get the latest commit hash on the 'main' branch of the base model
base_model_info = api.model_info(repo_id=base_model_name, revision="main")
base_model_commit_hash = base_model_info.sha

print(f"Base model: {base_model_name}")
print(f"Base model commit hash: {base_model_commit_hash}")

Base model: unsloth/qwen3-1.7B
Base model commit hash: 6262b50d6c1f8ee5e4ac750d710c33603bfc2a0c
